Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = ""
IMMATRICULATION_NUMBER = ""

---

# Exercise 8: Grammar-based compression of trees into DAGs 

Goals of this exercise:  
* Get a deeper understanding of grammar-based compression of trees into DAGs 
* Get a deeper understanding of navigation in grammars
* Get a deeper understanding of pruning

## 1. Construct a grammar of binary rules from an input string     

Implement a function <code>string2grammar</code> that can be called by 

<code>newGrammar = string2grammar(inputString)</code>

and that reads an input string representing a binary tree and that returns a grammar of binary DAG rules for that tree. A binary DAG rule is a dictionary element of the type ```'L':'t(F,N)'``` where L is a nonterminal, t is a terminal, and F and N are a terminal or a nonterminal or ‘-‘. 

For example, 

<code>newGrammar = string2grammar( “p(-,q(r(-,-),r(-,-)))” )</code>

should return a grammar with the 3 binary DAG rules:  <code>{"A":"r(-,-)" , "B":"q(A,A)" , "C":"p(-,B)"}</code> . 
Note that the same rule A shall be called twice in the right-hand side of rule B. 

To simplify your implementation, you shall assume the following: 
* the input string represents a syntactically correct binary tree, 
* the input string contains only lower case letters and the symbols   ‘(‘  ,   ‘)’  ,   ’,’  ,   and   ‘-‘
* lower case letters represent terminal nodes of arity 2 (i.e., have a first child and a next sibling) 
* the letter ‘-‘ represents a terminal node of arity 0 (having neither a first child nor a next sibling) 
* nonterminals consist of a single capital letter A,B,C, … only. 

There are several possible solutions to implement such a function. Use the hints to get a short solution. 

<b>Hint 1: </b>
The grammar of binary DAG rules represents a DAG of the binary tree. You add a node to the DAG (= you add a rule to the grammar) when you know the node’s name, its first child, and its next sibling. 
Therefore, you must make sure that you have inserted the first child and the next sibling into the DAG (i.e., have created the binary rules for the first-child sub-tree and for the next-sibling sub-tree), before you add the node itself to the DAG (=add the rule for this DAG node to the grammar). 

<b>Hint 2:  </b>
One possibility is to search for innermost pairs of ‘(‘ and ‘)’, i.e. to start generating binary rules from the substrings that are enclosed by an innermost pair and preceded by a terminal. You can find the innermost pair by searching the first ')' and then go back to the position of the corresponding '('. As it is the innermost pair, no other pair of '(' and ')' is in between.

<b>Hint 3:  </b>
Once, you have generated a rule ```'L':'t(F,N)'```, it may simplify your program if you substitute all occurrences of ```”t(F,N)”``` in the input string by L at once. You can consider to modify your method replaceMFD from the StringRePair exercise. 

<b>Hint 4:</b> 
Using standard Python functions (find,append,replace,…) a short Python solution takes only 9 lines.


In [2]:
def string2grammar(S):                 # compute a DAG rule grammar D from an input string S
    D = {}
    current_nonterminal = 'A'
    while len(S) > 1:
        rb = S.find(')')
        lb = S.rfind('(', 0, rb)
        if lb == -1:
            return None
        rule = S[lb-1:rb+1]
        D[current_nonterminal] = rule
        S = S.replace(rule, current_nonterminal)
        current_nonterminal = chr(ord(current_nonterminal) + 1)
    return D

In [3]:
newGrammar = string2grammar("p(-,q(r(-,-),r(-,-)))")
assert newGrammar == {"A":"r(-,-)" , "B":"q(A,A)" , "C":"p(-,B)"}

## 2. Get terminal of grammar path in a non-pruned DAG grammar     

Implement a function <code>getTerminal</code> that can be called by 

<code>c = getTerminal(newGrammar,grammarPath)</code>

and that reads a grammar and a grammar path and returns the terminal symbol represented by that grammar path. 
Each grammar path is a list of tuples (nt,p) where nt is a non-terminal symbol and p is the position within the right-hand side of the rule defined by the non-terminal symbol nt.

For example, for the grammar 

<code>{"A":"r(-,-)" , "B":"q(A,A)" , "C":"p(-,B)"}</code>

 the following calls of getTerminal should give the following results: 
<code>
getTerminal ( newGrammar, [("C",0)] ) = ‘p’ 
getTerminal ( newGrammar, [("C",2)) = ‘-‘ 
getTerminal ( newGrammar, [("C",4),("B",0)]] ) = ‘q’ 
getTerminal ( newGrammar, [("C",4),("B",2),("A",0)] ) = ‘r’ 
getTerminal ( newGrammar, [("C",4),("B",2),("A",2)] ) = ‘-‘ 
getTerminal ( newGrammar, [("C",4),("B",2),("A",4)] ) = ‘-‘ 
getTerminal ( newGrammar, [("C",4),("B",4),("A",0)] ) = ‘r’ 
getTerminal ( newGrammar, [("C",4),("B",4),("A",2)] ) = ‘-‘ 
getTerminal ( newGrammar, [("C",4),("B",4),("A",4)] ) = ‘-‘ 
</code>

In [27]:
def getTerminal(G,GP):
    t = None
    GP = GP.copy()
    while GP:
        nt,p = GP.pop(0)
        if t is not None:
            assert t == nt
        rule = G[nt]
        t = rule[p]
        # print(f"{nt=},{p=},{rule=},{t=}")
    return t


In [28]:
newGrammar = {"A":"r(-,-)" , "B":"q(A,A)" , "C":"p(-,B)"}

assert getTerminal ( newGrammar, [("C",0)] ) == "p"
assert getTerminal ( newGrammar, [("C",2)] ) == "-" 
assert getTerminal ( newGrammar, [("C",4),("B",0)]) == "q" 
assert getTerminal ( newGrammar, [("C",4),("B",2),("A",0)] ) == "r"
assert getTerminal ( newGrammar, [("C",4),("B",2),("A",2)] ) == "-" 
assert getTerminal ( newGrammar, [("C",4),("B",2),("A",4)] ) == "-" 
assert getTerminal ( newGrammar, [("C",4),("B",4),("A",0)] ) == "r"
assert getTerminal ( newGrammar, [("C",4),("B",4),("A",2)] ) == "-"
assert getTerminal ( newGrammar, [("C",4),("B",4),("A",4)] ) == "-"

## 3. Navigation to first-child or to next-sibling     

Implement a function <code>nav</code> that can be called by 

<code>nav(newGrammar,grammarPath,fcns)</code>

and that reads a grammar, a complete grammar path, and a navigation direction (‘fc’ or ‘ns’) and that **modifies the grammar path as a side effect**, such that it points to the first child or to the next sibling depending on the value of the parameter fcns. Furthermore, the new actual grammar path and the terminal node referenced by the grammar path shall be returned. 

For example, for the grammar given above, if the grammar path is initially <code>grammarPath = [("C",0)]</code>, i.e. pointing to the tree’s root node labeled ‘p’, the following sequence of calls of nav shall produce the following output:

<code>
1st call nav(newGrammar,grammarPath,’ns’)	shall return: 	( [("C",4),("B",0)] , ‘q’ )
2nd call nav(newGrammar,grammarPath,’fc’)	shall return: 	( [("C",4),("B",2),("A",0)] , ‘r’ )   
3rd call nav(newGrammar,grammarPath,’ns’)	shall return: 	( [("C",4),("B",2),("A",4)] , ‘-‘ )
</code>

<b>Hint:</b> To simplify your code, you shall assume that the grammar paths are always correct complete grammar paths referencing a terminal node.  

<b>Hint 2:</b> You can assume that your grammar is not pruned, i.e., that the first-child as well as the next-sibling consists of a characters letter only. I.e., you will not have a grammar like <code>{"C":"r(q(-,-),s(-,-))"}</code> where the first-child and the next-sibling of the node with label r span 6 characters.

**Hint 3:** You can use the methods 
- list.append(element) to append to the grammar path
- list.pop(pos) to delete from the grammar path


In [29]:
def nav(G,GP,fcns):                    # go to fc or ns of terminal referenced by grammar path
    t = None
    GP_copy = GP.copy()
    rule = GP[-1]
    while GP_copy:
        nt,p = GP_copy.pop(0)
        if t is not None:
            assert t == nt
        rule = G[nt]
        t = rule[p]
    if fcns == "fc":
        t = rule[2]
        # GP[-1][1] = 2
        GP[-1] = (GP[-1][0],2)
    elif fcns == "ns":
        t = rule[4]
        # GP[-1][1] = 4
        GP[-1] = (GP[-1][0],4)
    if str.upper(t) == t and t != "-":
        GP.append((t,0))
        t = G[GP[-1][0]][GP[-1][1]]
    print(f"{nt=},{p=},{rule=},{t=},{fcns=},{GP=}")
    return GP,t

        


In [30]:
newGrammar = {"A":"r(-,-)" , "B":"q(A,A)" , "C":"p(-,B)"}
grammarPath = [("C",0)]

assert nav(newGrammar,grammarPath,'ns') == ([("C",4),("B",0)], 'q')
assert nav(newGrammar,grammarPath,'fc') == ([("C",4),("B",2),("A",0)], 'r')   
assert nav(newGrammar,grammarPath,'ns') == ([("C",4),("B",2),("A",4)], '-')

newGrammar = {"A":"r(-,-)" , "B":"q(A,A)" , "C":"p(A,B)"}
grammarPath = [("C",0)]

assert nav(newGrammar,grammarPath,'fc') == ([("C",2),("A",0)], 'r')


nt='C',p=0,rule='p(-,B)',t='q',fcns='ns',GP=[('C', 4), ('B', 0)]
nt='B',p=0,rule='q(A,A)',t='r',fcns='fc',GP=[('C', 4), ('B', 2), ('A', 0)]
nt='A',p=0,rule='r(-,-)',t='-',fcns='ns',GP=[('C', 4), ('B', 2), ('A', 4)]
nt='C',p=0,rule='p(A,B)',t='r',fcns='fc',GP=[('C', 2), ('A', 0)]


## 4. Navigation back to previous node, i.e. reverse of first-child and next-sibling   

Implement a function <code>back</code> that can be called by 

<code>back(newGrammar,grammarPath)</code>

and that reads a grammar and a complete grammar path (which represents a terminal t in the tree) and that **modifies the grammar path as a side effect**, such that the grammar path represents the previous node in the tree (i.e. that node of which t is a first-child or a next-sibling). Furthermore, the new actual grammar path and the terminal node referenced by the grammar path shall be returned. 

For example, for the grammar given above, if the grammar path is initially <code>grammarPath = [("C",4),("B",2),("A",4)]</code>, i.e. pointing to the next-sibling (‘-‘) of the first-child (‘r’) of the next-sibling (‘q’) of the tree’s root node (‘p’), the following sequence of calls of back shall modify grammarPath as follows: 
`{"A":"r(-,-)" , "B":"q(A,A)" , "C":"p(-,B)"}`
<code>
after 1st call of back(newGrammar,grammarPath) shall return ([("C",4),("B",2),("A",0)], 'r') , 
after 2nd call of back(newGrammar,grammarPath) shall return ([("C",4),("B",0)], 'q')  	 , and	
after 3rd call of back(newGrammar,grammarPath) shall return ([("C",0)], 'p') 		 .
</code>

<b>Hint:</b> Look at the examples to see what special treatment is needed, if the current value of grammarPath references the first node (i.e. at position 0) of a rule. 

<b>Hint 2:</b> You can assume that your grammar is not pruned, i.e., that the first-child as well as the next-sibling consists of a characters letter only. I.e., you will not have a grammar like <code>{"C":"r(q(-,-),s(-,-))"}</code> where the first-child and the next-sibling of the node with label r span 6 characters.


**Hint 3:** You can use the methods 
- list.append(element) to append to the grammar path
- list.pop(pos) to delete from the grammar path

In [31]:
def back(G,GP):                        # go back to previous-sibling or parent of fc
    top = GP.pop()
    if top[1] == 0:
        GP[-1] = (GP[-1][0],0)
    else:
        GP.append((top[0],0))
    nt = GP[-1][0]
    letter = G[nt][0]
    print(f"{GP=} {letter=}")
    return GP,letter

In [32]:
newGrammar = {"A":"r(-,-)" , "B":"q(A,A)" , "C":"p(-,B)"}
grammarPath = [("C",4),("B",2),("A",4)]

assert back(newGrammar,grammarPath) == ([("C",4),("B",2),("A",0)], 'r')
assert back(newGrammar,grammarPath) == ([("C",4),("B",0)], 'q')   
assert back(newGrammar,grammarPath) == ([("C",0)], 'p')         


def G2allGPs(G):                                    # print all grammar paths of the tree represented by grammar G
    output = []
    GP = [(max(G),0)]                               # grammar path GP represents the first terminal of the start rule
    output.append((GP.copy(),getTerminal(G,GP)))    # print grammar path and referenced terminal symbol
    G2GPs(G,GP,output)                              # print all grammar paths of G and referenced terminals
    return output
  
def G2GPs(G,GP,output):                           # print all grammar paths of the subtree represented by GP
    if getTerminal(G,GP) != '-' :                 # if I am not at a leaf
        newGP = nav(G,GP,'fc')                    # go to fc and show GP and referenced terminal
        output.append((newGP[0].copy(),newGP[1]))
        G2GPs(G,GP,output)                        # traverse fc's sub-tree
        back(G,GP)                                # go back to root (of subtree) 
        newGP = nav(G,GP,'ns')                    # go to ns and show GP and referenced terminal 
        output.append((newGP[0].copy(),newGP[1]))
        G2GPs(G,GP,output)                        # traverse ns's sub-tree
        back(G,GP)                                # go back to root (of subtree)


assert G2allGPs(newGrammar)== [([('C', 0)], 'p'), 
        ([('C', 2)], '-'), 
        ([('C', 4), ('B', 0)], 'q'), 
        ([('C', 4), ('B', 2), ('A', 0)], 'r'), 
        ([('C', 4), ('B', 2), ('A', 2)], '-'), 
        ([('C', 4), ('B', 2), ('A', 4)], '-'),
        ([('C', 4), ('B', 4), ('A', 0)], 'r'), 
        ([('C', 4), ('B', 4), ('A', 2)], '-'), 
        ([('C', 4), ('B', 4), ('A', 4)], '-')]



GP=[('C', 4), ('B', 2), ('A', 0)] letter='r'
GP=[('C', 4), ('B', 0)] letter='q'
GP=[('C', 0)] letter='p'
nt='C',p=0,rule='p(-,B)',t='-',fcns='fc',GP=[('C', 2)]
GP=[('C', 0)] letter='p'
nt='C',p=0,rule='p(-,B)',t='q',fcns='ns',GP=[('C', 4), ('B', 0)]
nt='B',p=0,rule='q(A,A)',t='r',fcns='fc',GP=[('C', 4), ('B', 2), ('A', 0)]
nt='A',p=0,rule='r(-,-)',t='-',fcns='fc',GP=[('C', 4), ('B', 2), ('A', 2)]
GP=[('C', 4), ('B', 2), ('A', 0)] letter='r'
nt='A',p=0,rule='r(-,-)',t='-',fcns='ns',GP=[('C', 4), ('B', 2), ('A', 4)]
GP=[('C', 4), ('B', 2), ('A', 0)] letter='r'
GP=[('C', 4), ('B', 0)] letter='q'
nt='B',p=0,rule='q(A,A)',t='r',fcns='ns',GP=[('C', 4), ('B', 4), ('A', 0)]
nt='A',p=0,rule='r(-,-)',t='-',fcns='fc',GP=[('C', 4), ('B', 4), ('A', 2)]
GP=[('C', 4), ('B', 4), ('A', 0)] letter='r'
nt='A',p=0,rule='r(-,-)',t='-',fcns='ns',GP=[('C', 4), ('B', 4), ('A', 4)]
GP=[('C', 4), ('B', 4), ('A', 0)] letter='r'
GP=[('C', 4), ('B', 0)] letter='q'
GP=[('C', 0)] letter='p'


## 5. Count how often each rule is called   

Implement a function <code>computeCalls</code> that can be called by

<code>calls = computeCalls(grammar)</code>

and that returns a dictionary with a mapping of a grammar rule's non-terminal symbol to the number of direct calls of that grammar rule for each grammar rule. 

For example, for the above given grammar with the rules  <code>{"A":"r(-,-)" , "B":"q(A,A)" , "C":"p(-,B)"}</code>, the returned dictionary should contain the values <code>{'A': 2, 'B': 1, 'C': 0}</code> representing that A is called twice, B is called once, and C is not called. 


In [37]:
def computeCalls(D):                   # how often is each rule D[i] directly called? 
    calls = {}
    for key, _ in D.items():
        count = 0
        for _, value in D.items():
            # if key in value:
            #     count += 1
            count += value.count(key)
        calls[key] = count
    # print(f"{calls=}")
    return calls

In [38]:
newGrammar = {"A":"r(-,-)" , "B":"q(A,A)" , "C":"p(-,B)"}

assert computeCalls(newGrammar)=={'A': 2, 'B': 1, 'C': 0}

calls={'A': 2, 'B': 1, 'C': 0}


## 6. Prune the grammar    

Implement a function <code>prune</code> that can be called by

<code>prunedGrammar = prune(grammar,calls)</code>

that returns a pruned grammar. A pruned grammar inlines each rule of the grammar that is called exactly once. To inline a rule means to replace its call by its right-hand-side and to delete this rule from the grammar. 

For example, for a grammar with the rules <code>{"A":"r(-,-)" , "B":"q(A,A)" , "C":"p(-,B)"}</code>,
the pruned grammar is:  

<code>{'A': 'r(-,-)', 'C': 'p(-,q(A,A))'}</code>. 

<b>Hint</b>: depending on the implementation of your grammar, it may be useful to copy the grammar or to modify the grammar before pruning it. 


In [39]:
    def prune(D):                          # prune all grammar rules that are called exactly once
        calls = computeCalls(D)
        to_prune = [ key for key, value in calls.items() if value == 1 ]
        for key in to_prune:
            # inline
            for k in D.keys():
                D[k] = D[k].replace(key, D[key])
            del D[key]
        return D


In [40]:
newGrammar = {"A":"r(-,-)" , "B":"q(A,A)" , "C":"p(-,B)"}

assert prune(newGrammar)=={'A': 'r(-,-)', 'C': 'p(-,q(A,A))'}

calls={'A': 2, 'B': 1, 'C': 0}


In [41]:
### ROUNDTRIP RANDOM TESTS

import random

def makeRandTree(trees):
    labels = ['p','q','r','s']
    out = random.choice(labels) + "("
    fc = "-"
    if(random.randint(1,2)==1): 
        fc = makeRandTree(trees)
    out = out + fc + ","
    ns = "-"
    if(random.randint(1,4)<=2): 
        ns = makeRandTree(trees)
    out = out + ns + ")"
    if(len(trees)>=26 and len(fc)>len(ns)):return fc
    elif (len(trees)>=26): return ns
    trees[out] = 0
    return out

def decomp(G):                            
    GP = [(max(G),0)]                     
    output = decompRec(G,GP)      
    return output
  
def decompRec(G,GP):                         
    output = getTerminal(G,GP)
    if getTerminal(G,GP) != '-' :            
        nav(G,GP,'fc')                   
        output = output + "(" + decompRec(G,GP)
        back(G,GP)                       
        nav(G,GP,'ns')                   
        output = output + "," + decompRec(G,GP)
        back(G,GP)                       
        output = output + ")"
    return output

def testDAG():
    trees = {}
    tree = makeRandTree(trees)
    grammar = string2grammar(tree)
    if(ord(max(grammar))>ord('A')+25):
        return
    assert tree == decomp(grammar), "Combination comp/decomp failed for tree\n %s\n and grammar \n%s" % (tree, grammar)
    prunedGrammar = prune(grammar)

for i in range(25):
    testDAG()
    
print("All tests were performed successfully!")

nt='D',p=0,rule='r(B,C)',t='q',fcns='fc',GP=[('D', 2), ('B', 0)]
nt='B',p=0,rule='q(-,A)',t='-',fcns='fc',GP=[('D', 2), ('B', 2)]
GP=[('D', 2), ('B', 0)] letter='q'
nt='B',p=0,rule='q(-,A)',t='s',fcns='ns',GP=[('D', 2), ('B', 4), ('A', 0)]
nt='A',p=0,rule='s(-,-)',t='-',fcns='fc',GP=[('D', 2), ('B', 4), ('A', 2)]
GP=[('D', 2), ('B', 4), ('A', 0)] letter='s'
nt='A',p=0,rule='s(-,-)',t='-',fcns='ns',GP=[('D', 2), ('B', 4), ('A', 4)]
GP=[('D', 2), ('B', 4), ('A', 0)] letter='s'
GP=[('D', 2), ('B', 0)] letter='q'
GP=[('D', 0)] letter='r'
nt='D',p=0,rule='r(B,C)',t='p',fcns='ns',GP=[('D', 4), ('C', 0)]
nt='C',p=0,rule='p(-,-)',t='-',fcns='fc',GP=[('D', 4), ('C', 2)]
GP=[('D', 4), ('C', 0)] letter='p'
nt='C',p=0,rule='p(-,-)',t='-',fcns='ns',GP=[('D', 4), ('C', 4)]
GP=[('D', 4), ('C', 0)] letter='p'
GP=[('D', 0)] letter='r'
calls={'A': 1, 'B': 1, 'C': 1, 'D': 0}
nt='X',p=0,rule='p(A,W)',t='r',fcns='fc',GP=[('X', 2), ('A', 0)]
nt='A',p=0,rule='r(-,-)',t='-',fcns='fc',GP=[('X', 2), ('A', 2)]
G